### Project Title

### Team Members

### Overview

### Motivation

### Data Sources

### Data Description

### Data Manipulation

### Data Visualization

In [318]:
# install missing packages
#%pip install pandas numpy matplotlib seaborn

# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [319]:
# load data
hospital_general_info = pd.read_csv("../dataset/Hospital_General_Information.csv")
promoting_interoperability = pd.read_csv("../dataset/Promoting_Interoperability_Hospital.csv")

In [320]:
print("Hospital General Information Data:")
print(hospital_general_info.head())

Hospital General Information Data:
  Facility ID                    Facility Name                     Address  \
0      010001  SOUTHEAST HEALTH MEDICAL CENTER      1108 ROSS CLARK CIRCLE   
1      010005         MARSHALL MEDICAL CENTERS  2505 U S HIGHWAY 431 NORTH   
2      010006     NORTH ALABAMA MEDICAL CENTER         1701 VETERANS DRIVE   
3      010007         MIZELL MEMORIAL HOSPITAL               702 N MAIN ST   
4      010008      CRENSHAW COMMUNITY HOSPITAL         101 HOSPITAL CIRCLE   

  City/Town State  ZIP Code County/Parish Telephone Number  \
0    DOTHAN    AL     36301       HOUSTON   (334) 793-8701   
1      BOAZ    AL     35957      MARSHALL   (256) 593-8310   
2  FLORENCE    AL     35630    LAUDERDALE   (256) 768-8400   
3       OPP    AL     36467     COVINGTON   (334) 493-3541   
4   LUVERNE    AL     36049      CRENSHAW   (334) 335-3374   

          Hospital Type                           Hospital Ownership  ...  \
0  Acute Care Hospitals  Government - Hospital

In [321]:
sh = hospital_general_info.shape
print(f'hospital_general dataset has {sh[0]} rows and {sh[1]}columns')

hospital_general dataset has 5381 rows and 38columns


In [322]:
# Briefly review 
hospital_general_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5381 entries, 0 to 5380
Data columns (total 38 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   Facility ID                                       5381 non-null   object 
 1   Facility Name                                     5381 non-null   object 
 2   Address                                           5381 non-null   object 
 3   City/Town                                         5381 non-null   object 
 4   State                                             5381 non-null   object 
 5   ZIP Code                                          5381 non-null   int64  
 6   County/Parish                                     5381 non-null   object 
 7   Telephone Number                                  5381 non-null   object 
 8   Hospital Type                                     5381 non-null   object 
 9   Hospital Ownership 

In [323]:
# remove the columns related to Footnote
# drop columns address, zip code, phone number
exclude_pattern = 'Footnote'
cols_to_drop = hospital_general_info.columns[hospital_general_info.columns.str.contains(exclude_pattern, case=False)]
df_hospital_filtered = hospital_general_info.drop(columns=cols_to_drop)

exclude_column = [ 'Address', 'ZIP Code', 'Telephone Number']
df_hospital_filtered = df_hospital_filtered.drop(columns=exclude_column)

In [324]:
#replace abbreviation to state name

state_abbrev = {
    'AL': 'Alabama',
    'AK': 'Alaska',
    'AZ': 'Arizona',
    'AR': 'Arkansas',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'HI': 'Hawaii',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'IA': 'Iowa',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'ME': 'Maine',
    'MD': 'Maryland',
    'MA': 'Massachusetts',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MS': 'Mississippi',
    'MO': 'Missouri',
    'MT': 'Montana',
    'NE': 'Nebraska',
    'NV': 'Nevada',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NY': 'New York',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VT': 'Vermont',
    'VA': 'Virginia',
    'WA': 'Washington',
    'WV': 'West Virginia',
    'WI': 'Wisconsin',
    'WY': 'Wyoming',
    'DC': 'District of Columbia',    
    'PR': 'Puerto Rico',
    'GU': 'Guam',
    'VI': 'U.S. Virgin Islands',
    'AS': 'American Samoa',
    'MP': 'Northern Mariana Islands'
}
df_hospital_filtered ['State']= df_hospital_filtered['State'].map(state_abbrev)


In [325]:
# Replace specified strings with np.nan in the each column
hospital_col_lst = df_hospital_filtered.columns.to_list()
replace_string = ['Not Available', 'N']
for _ in hospital_col_lst: 
    df_hospital_filtered[_] = df_hospital_filtered[_].\
        replace(replace_string, np.nan)

#

In [326]:
# Find the missing value
hospital_col_lst = df_hospital_filtered.columns.to_list()
print('The columns with missing value : ')
missing_col_lst = []
for _ in hospital_col_lst:
    missing_value = df_hospital_filtered[_].isnull().sum()
    if missing_value:
        print(f'{_} : {missing_value} missing values')
        missing_col_lst.append(_)

The columns with missing value : 
Meets criteria for birthing friendly designation : 3153 missing values
Hospital overall rating : 2509 missing values
MORT Group Measure Count : 811 missing values
Count of Facility MORT Measures : 1726 missing values
Count of MORT Measures Better : 1726 missing values
Count of MORT Measures No Different : 1726 missing values
Count of MORT Measures Worse : 1726 missing values
Safety Group Measure Count : 811 missing values
Count of Facility Safety Measures : 2017 missing values
Count of Safety Measures Better : 2017 missing values
Count of Safety Measures No Different : 2017 missing values
Count of Safety Measures Worse : 2017 missing values
READM Group Measure Count : 811 missing values
Count of Facility READM Measures : 1098 missing values
Count of READM Measures Better : 1098 missing values
Count of READM Measures No Different : 1098 missing values
Count of READM Measures Worse : 1098 missing values
Pt Exp Group Measure Count : 811 missing values
Cou

In [327]:
# The column name : 'Meets criteria for birthing friendly designation'
# This columns is binary, so put 'N' in no value.
df_hospital_filtered['Meets criteria for birthing friendly designation'] = df_hospital_filtered\
    ['Meets criteria for birthing friendly designation'].fillna('N')
df_hospital_filtered['Meets criteria for birthing friendly designation'].isna().value_counts()

Meets criteria for birthing friendly designation
False    5381
Name: count, dtype: int64

In [328]:
df_hospital_filtered_2 = df_hospital_filtered.copy()

In [329]:
df_hospital_column_lst = df_hospital_filtered_2.columns.to_list()
print(df_hospital_column_lst)
filter_index = df_hospital_column_lst.index(
    'Meets criteria for birthing friendly designation')
for col in df_hospital_column_lst[filter_index+1 : ]:
    # Convert column to float for calculation
    df_hospital_filtered_2[col] = pd.to_numeric(df_hospital_filtered_2[col], errors='coerce')
    avg = df_hospital_filtered_2[col].mean()
    df_hospital_filtered_2[col] = df_hospital_filtered_2[col].fillna(avg).round(2)
    if df_hospital_filtered_2[col].isna().sum() == 0:
        print(f'Removed nulls by entering average in column {col} successfully')
    else:
        print(f'There are still null values in column {col}')
df_hospital_filtered_2.head(n=1)

['Facility ID', 'Facility Name', 'City/Town', 'State', 'County/Parish', 'Hospital Type', 'Hospital Ownership', 'Emergency Services', 'Meets criteria for birthing friendly designation', 'Hospital overall rating', 'MORT Group Measure Count', 'Count of Facility MORT Measures', 'Count of MORT Measures Better', 'Count of MORT Measures No Different', 'Count of MORT Measures Worse', 'Safety Group Measure Count', 'Count of Facility Safety Measures', 'Count of Safety Measures Better', 'Count of Safety Measures No Different', 'Count of Safety Measures Worse', 'READM Group Measure Count', 'Count of Facility READM Measures', 'Count of READM Measures Better', 'Count of READM Measures No Different', 'Count of READM Measures Worse', 'Pt Exp Group Measure Count', 'Count of Facility Pt Exp Measures', 'TE Group Measure Count', 'Count of Facility TE Measures']
Removed nulls by entering average in column Hospital overall rating successfully
Removed nulls by entering average in column MORT Group Measure Co

,Facility ID,Facility Name,City/Town,State,County/Parish,Hospital Type,Hospital Ownership,Emergency Services,Meets criteria for birthing friendly designation,Hospital overall rating,...,Count of Safety Measures Worse,READM Group Measure Count,Count of Facility READM Measures,Count of READM Measures Better,Count of READM Measures No Different,Count of READM Measures Worse,Pt Exp Group Measure Count,Count of Facility Pt Exp Measures,TE Group Measure Count,Count of Facility TE Measures
0,010001,SOUTHEAST HEALTH MEDICAL CENTER,DOTHAN,Alabama,HOUSTON,Acute Care Hospitals,Government - Hospital District or Authority,Yes,Y,4.0,...,0.0,11.0,11.0,0.0,11.0,0.0,8.0,8.0,12.0,11.0


In [330]:

df_hospital_clean = df_hospital_filtered_2.copy()

In [331]:
# First two rows of dataset
df_hospital_clean.head(n=2)

,Facility ID,Facility Name,City/Town,State,County/Parish,Hospital Type,Hospital Ownership,Emergency Services,Meets criteria for birthing friendly designation,Hospital overall rating,...,Count of Safety Measures Worse,READM Group Measure Count,Count of Facility READM Measures,Count of READM Measures Better,Count of READM Measures No Different,Count of READM Measures Worse,Pt Exp Group Measure Count,Count of Facility Pt Exp Measures,TE Group Measure Count,Count of Facility TE Measures
0,010001,SOUTHEAST HEALTH MEDICAL CENTER,DOTHAN,Alabama,HOUSTON,Acute Care Hospitals,Government - Hospital District or Authority,Yes,Y,4.0,...,0.0,11.0,11.0,0.0,11.0,0.0,8.0,8.0,12.0,11.0
1,010005,MARSHALL MEDICAL CENTERS,BOAZ,Alabama,MARSHALL,Acute Care Hospitals,Government - Hospital District or Authority,Yes,N,3.0,...,0.0,11.0,9.0,0.0,8.0,1.0,8.0,8.0,12.0,12.0


In [332]:
sh = df_hospital_clean.shape
print(f'The cleaned hospital general information dataset has {sh[0]}rows and \
{sh[1]}column')

The cleaned hospital general information dataset has 5381rows and 29column


In [333]:
print('Basic statistic analysis')
df_hospital_clean.describe()

Basic statistic analysis


,Hospital overall rating,MORT Group Measure Count,Count of Facility MORT Measures,Count of MORT Measures Better,Count of MORT Measures No Different,Count of MORT Measures Worse,Safety Group Measure Count,Count of Facility Safety Measures,Count of Safety Measures Better,Count of Safety Measures No Different,Count of Safety Measures Worse,READM Group Measure Count,Count of Facility READM Measures,Count of READM Measures Better,Count of READM Measures No Different,Count of READM Measures Worse,Pt Exp Group Measure Count,Count of Facility Pt Exp Measures,TE Group Measure Count,Count of Facility TE Measures
count,5381.000000,5381.0,5381.000000,5381.000000,5381.000000,5381.000000,5381.0,5381.000000,5381.000000,5381.000000,5381.000000,5381.0,5381.000000,5381.000000,5381.000000,5381.000000,5381.0,5381.0,5381.0,5381.000000
mean,3.081345,7.0,4.378904,0.229322,4.022026,0.127556,8.0,4.739933,0.898402,3.701524,0.140007,11.0,6.260636,0.280513,5.536131,0.446032,8.0,8.0,12.0,7.960966
std,0.803507,0.0,1.754744,0.556029,1.629655,0.338271,0.0,1.950856,0.857032,1.464141,0.313588,0.0,2.817619,0.534092,2.496596,0.715965,0.0,0.0,0.0,2.726853
min,1.000000,7.0,1.000000,0.000000,0.000000,0.000000,8.0,1.000000,0.000000,0.000000,0.000000,11.0,1.000000,0.000000,0.000000,0.000000,8.0,8.0,12.0,1.000000
25%,3.000000,7.0,3.000000,0.000000,3.000000,0.000000,8.0,4.000000,0.000000,3.000000,0.000000,11.0,4.000000,0.000000,4.000000,0.000000,8.0,8.0,12.0,6.000000
50%,3.080000,7.0,4.380000,0.000000,4.020000,0.000000,8.0,4.740000,0.900000,3.700000,0.000000,11.0,6.260000,0.000000,5.540000,0.000000,8.0,8.0,12.0,8.000000
75%,3.080000,7.0,6.000000,0.230000,5.000000,0.130000,8.0,6.000000,1.000000,4.000000,0.140000,11.0,8.000000,0.280000,7.000000,0.450000,8.0,8.0,12.0,10.000000
max,5.000000,7.0,7.000000,7.000000,7.000000,5.000000,8.0,8.000000,6.000000,8.000000,3.000000,11.0,11.000000,5.000000,11.000000,7.000000,8.0,8.0,12.0,12.000000


In [334]:
df_hospital_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5381 entries, 0 to 5380
Data columns (total 29 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   Facility ID                                       5381 non-null   object 
 1   Facility Name                                     5381 non-null   object 
 2   City/Town                                         5381 non-null   object 
 3   State                                             5381 non-null   object 
 4   County/Parish                                     5381 non-null   object 
 5   Hospital Type                                     5381 non-null   object 
 6   Hospital Ownership                                5381 non-null   object 
 7   Emergency Services                                5381 non-null   object 
 8   Meets criteria for birthing friendly designation  5381 non-null   object 
 9   Hospital overall ra

# Promoting dataset


In [335]:
print("Promoting Interoperability Data:")
print(promoting_interoperability.head())

Promoting Interoperability Data:
   Facility ID                    Facility Name                     Address  \
0        10001  SOUTHEAST HEALTH MEDICAL CENTER      1108 ROSS CLARK CIRCLE   
1        10005         MARSHALL MEDICAL CENTERS  2505 U S HIGHWAY 431 NORTH   
2        10006     NORTH ALABAMA MEDICAL CENTER         1701 VETERANS DRIVE   
3        10007         MIZELL MEMORIAL HOSPITAL               702 N MAIN ST   
4        10008      CRENSHAW COMMUNITY HOSPITAL         101 HOSPITAL CIRCLE   

  City/Town State  ZIP Code County/Parish Telephone Number         CEHRT ID  \
0    DOTHAN    AL     36301       HOUSTON   (334) 793-8701  0015CAN28DKT47C   
1      BOAZ    AL     35957      MARSHALL   (256) 593-8310  0015C0HAX4ESQ0D   
2  FLORENCE    AL     35630    LAUDERDALE   (256) 768-8400  0015CW76TRC3SVN   
3       OPP    AL     36467     COVINGTON   (334) 493-3541  0015CFG3Q10HY2V   
4   LUVERNE    AL     36049      CRENSHAW   (334) 335-3374  0015CFG3Q10HY2V   

  Meets criteria 

In [336]:
sh = promoting_interoperability.shape
print(f'This dataset has {sh[0]} rows and{sh[1]} columns')

This dataset has 4591 rows and12 columns


In [337]:
promoting_interoperability.head(n=2)

,Facility ID,Facility Name,Address,City/Town,State,ZIP Code,County/Parish,Telephone Number,CEHRT ID,Meets criteria for promoting interoperability of EHRs,Start Date,End Date
0,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,0015CAN28DKT47C,Y,01/01/2023,12/31/2023
1,10005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,MARSHALL,(256) 593-8310,0015C0HAX4ESQ0D,Y,01/01/2023,12/31/2023


In [338]:
# select the needed columns by dropping the excluded columns
exclude_column = ['Address','ZIP Code',
                  'Telephone Number', 'Start Date', 'End Date' ]
df_promoting_filtered = promoting_interoperability.drop(columns=exclude_column)

In [339]:
#replace abbreviation to state name
df_promoting_filtered ['State']= df_promoting_filtered['State'].\
    map(state_abbrev)

In [340]:
promot_col_lst = df_promoting_filtered.columns.to_list()
df_promoting_filtered.isna().value_counts()

Facility ID  Facility Name  City/Town  State  County/Parish  CEHRT ID  Meets criteria for promoting interoperability of EHRs
False        False          False      False  False          False     False                                                    3678
                                                             True      True                                                      553
                                                             False     True                                                      360
Name: count, dtype: int64

In [341]:
# find missing value
print('The number of missing value ')
for col in promot_col_lst: 
    missing_value = df_promoting_filtered[col].isna().sum()
    if missing_value:
        print(f'{col} : {missing_value} missing value')

The number of missing value 
CEHRT ID : 553 missing value
Meets criteria for promoting interoperability of EHRs : 913 missing value


In [342]:
# fill "No data" if no value in CEHRT
df_promoting_filtered['CEHRT ID'] = df_promoting_filtered['CEHRT ID'].\
    fillna("No data")
assert df_promoting_filtered['CEHRT ID'].isna().sum() == 0

In [343]:
#column name :'Meets criteria for promoting interoperability of EHRs
# This column is binary. if no value, fill "N"
criteria = df_promoting_filtered\
    ['Meets criteria for promoting interoperability of EHRs']
criteria = criteria.fillna('N')

result = criteria.value_counts()
result

Meets criteria for promoting interoperability of EHRs
Y    3678
N     913
Name: count, dtype: int64

In [344]:
df_promoting_cleaned = df_promoting_filtered.copy()


In [345]:
df_promoting_cleaned.head(n=2)

,Facility ID,Facility Name,City/Town,State,County/Parish,CEHRT ID,Meets criteria for promoting interoperability of EHRs
0,10001,SOUTHEAST HEALTH MEDICAL CENTER,DOTHAN,Alabama,HOUSTON,0015CAN28DKT47C,Y
1,10005,MARSHALL MEDICAL CENTERS,BOAZ,Alabama,MARSHALL,0015C0HAX4ESQ0D,Y


In [346]:
df_promoting_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4591 entries, 0 to 4590
Data columns (total 7 columns):
 #   Column                                                 Non-Null Count  Dtype 
---  ------                                                 --------------  ----- 
 0   Facility ID                                            4591 non-null   int64 
 1   Facility Name                                          4591 non-null   object
 2   City/Town                                              4591 non-null   object
 3   State                                                  4591 non-null   object
 4   County/Parish                                          4591 non-null   object
 5   CEHRT ID                                               4591 non-null   object
 6   Meets criteria for promoting interoperability of EHRs  3678 non-null   object
dtypes: int64(1), object(6)
memory usage: 251.2+ KB
